In [9]:
import numpy as np
from Huffman_Code import HuffmanShaping, GeometricHuffmanCode
from HuffmanMatching.utils import GaussHermiteModulation, RandomWalkModulation, DiscretisedGaussianModulation
from HuffmanMatching import HuffmanTree
from scipy.io import loadmat
from scipy.io import savemat

In [10]:
def compare_python_matlab_distributions(py_list, mat_file, mat_key, sizes=None, atol=1e-8):
    """
    Compara listas de distribuições do Python com resultados salvos em arquivo .mat do MATLAB.
    py_list: lista de arrays do Python
    mat_file: caminho do arquivo .mat
    mat_key: chave do array no arquivo .mat
    sizes: lista de tamanhos (opcional, para print)
    atol: tolerância para comparação
    """
    mat = loadmat(mat_file)
    mat_results = mat[mat_key]

    def extract_matlab_cell_array(cell_array):
        return [np.squeeze(cell_array[0, i]) if cell_array.shape[0] == 1 else np.squeeze(cell_array[i, 0]) for i in range(cell_array.shape[1] if cell_array.shape[0] == 1 else cell_array.shape[0])]

    if isinstance(mat_results, np.ndarray) and mat_results.dtype == 'O':
        mat_list = extract_matlab_cell_array(mat_results)
    else:
        mat_list = [np.squeeze(mat_results)]

    for idx, (py_arr, mat_arr) in enumerate(zip(py_list, mat_list)):
        py_arr = np.array(py_arr).flatten()
        mat_arr = np.array(mat_arr).flatten()
        print(py_arr)
        print(mat_arr)
        iguais = np.allclose(py_arr, mat_arr, atol=atol)
        n_val = sizes[idx] if sizes is not None else idx
        print(f'N = {n_val}: Iguais? {iguais}')
        if not iguais:
            print('Python:', py_arr)
            print('MATLAB:', mat_arr)

In [11]:
def save_distributions_to_mat(distributions, filename, key='distributions'):
    """
    Salva uma lista de distribuições em um arquivo .mat para uso no MATLAB.
    distributions: lista de arrays numpy
    filename: nome do arquivo .mat
    key: nome da variável no arquivo (default: 'distributions')
    """
    mat_dict = {key: distributions}
    savemat(filename, mat_dict)

In [12]:
class results:
    def __init__(self):
        self.constellation_size = []
        self.distribution = []
        self.huffman_geometric = []

In [13]:
# parameters

variance = 1
start = 4
stop = 101
step = 2

### gausshermite

In [14]:
gausshermite_results = results()
for N in range(start, stop+1, step):
    gausshermite_results.constellation_size.append(N)
    ghm = GaussHermiteModulation(N, variance)
    p = ghm.probas
    s = ghm.alphas
    gausshermite_results.distribution.append(p)
    huffman_geometric = GeometricHuffmanCode(s, p).dyadic_distribution
    gausshermite_results.huffman_geometric.append(huffman_geometric)

In [15]:
#save_distributions_to_mat(gausshermite_results.distribution, 'distributions.mat')

### Binomial

In [16]:
binomial_results = results()
for N in range(start, stop+1, step):
    binomial_results.constellation_size.append(N)
    bm = RandomWalkModulation(N, variance)
    p = bm.probas
    s = bm.alphas
    binomial_results.distribution.append(p)
    huffman_geometric = GeometricHuffmanCode(s, p).dyadic_distribution
    binomial_results.huffman_geometric.append(huffman_geometric)

In [17]:
save_distributions_to_mat(binomial_results.distribution, 'distributions_binomial.mat')

### gaussian distribution

In [18]:
gaussian_results = results()
for N in range(start, stop+1, step):
    gaussian_results.constellation_size.append(N)
    gaussian = DiscretisedGaussianModulation(N, variance)
    p = gaussian.probas
    s = gaussian.alphas
    gaussian_results.distribution.append(p)
    huffman_geometric = GeometricHuffmanCode(s, p).dyadic_distribution
    gaussian_results.huffman_geometric.append(huffman_geometric)

In [19]:
# Salva os resultados em um arquivo mat
# save_distributions_to_mat(gaussian_results.distribution, 'distributions_gaussian.mat')

### Compara resultados

##### gauss hermite

In [20]:
compare_python_matlab_distributions(gausshermite_results.huffman_geometric, 'pghc_results.mat', 'pghc_results', gausshermite_results.constellation_size)

[0.  0.5 0.5 0. ]
[0.  0.5 0.5 0. ]
N = 4: Iguais? True
[0.    0.125 0.25  0.5   0.125 0.   ]
[0.    0.125 0.25  0.5   0.125 0.   ]
N = 6: Iguais? True
[0.    0.    0.125 0.25  0.5   0.125 0.    0.   ]
[0.    0.    0.125 0.25  0.5   0.125 0.    0.   ]
N = 8: Iguais? True
[0.      0.      0.03125 0.0625  0.25    0.5     0.125   0.03125 0.
 0.     ]
[0.      0.      0.03125 0.0625  0.25    0.5     0.125   0.03125 0.
 0.     ]
N = 10: Iguais? True
[0.     0.     0.     0.0625 0.125  0.25   0.25   0.25   0.0625 0.
 0.     0.    ]
[0.     0.     0.     0.0625 0.125  0.25   0.25   0.25   0.0625 0.
 0.     0.    ]
N = 12: Iguais? True
[0.     0.     0.     0.     0.0625 0.125  0.25   0.25   0.25   0.0625
 0.     0.     0.     0.    ]
[0.     0.     0.     0.     0.0625 0.125  0.25   0.25   0.25   0.0625
 0.     0.     0.     0.    ]
N = 14: Iguais? True
[0.       0.       0.       0.       0.015625 0.03125  0.125    0.25
 0.25     0.25     0.0625   0.015625 0.       0.       0.       0.      

#### gaussian discrete

In [21]:
compare_python_matlab_distributions(gaussian_results.huffman_geometric, 'pghc_results_gaussian.mat', 'pghc_results', gaussian_results.constellation_size)

[0.  0.5 0.5 0. ]
[0.  0.5 0.5 0. ]
N = 4: Iguais? True
[0.  0.  0.5 0.5 0.  0. ]
[0.  0.  0.5 0.5 0.  0. ]
N = 6: Iguais? True
[0.  0.  0.  0.5 0.5 0.  0.  0. ]
[0.  0.  0.  0.5 0.5 0.  0.  0. ]
N = 8: Iguais? True
[0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0. ]
[0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0. ]
N = 10: Iguais? True
[0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0. ]
[0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0. ]
N = 12: Iguais? True
[0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0. ]
[0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0. ]
N = 14: Iguais? True
[0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0. ]
[0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0. ]
N = 16: Iguais? True
[0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
[0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
N = 18: Iguais? True
[0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0.
 0.  0. ]
[0.  0.  0.  0.  

#### binomial

In [22]:
compare_python_matlab_distributions(binomial_results.huffman_geometric, 'pghc_results_binomial.mat', 'pghc_results', binomial_results.constellation_size)

[0.125 0.25  0.5   0.125]
[0.125 0.25  0.5   0.125]
N = 4: Iguais? True
[0.0625 0.125  0.25   0.25   0.25   0.0625]
[0.0625 0.125  0.25   0.25   0.25   0.0625]
N = 6: Iguais? True
[0.015625 0.03125  0.125    0.25     0.25     0.25     0.0625   0.015625]
[0.015625 0.03125  0.125    0.25     0.25     0.25     0.0625   0.015625]
N = 8: Iguais? True
[0.      0.03125 0.0625  0.125   0.25    0.25    0.125   0.125   0.03125
 0.     ]
[0.      0.03125 0.0625  0.125   0.25    0.25    0.125   0.125   0.03125
 0.     ]
N = 10: Iguais? True
[0.        0.0078125 0.015625  0.0625    0.125     0.25      0.25
 0.125     0.125     0.03125   0.0078125 0.       ]
[0.        0.0078125 0.015625  0.0625    0.125     0.25      0.25
 0.125     0.125     0.03125   0.0078125 0.       ]
N = 12: Iguais? True
[0.         0.00195312 0.00390625 0.015625   0.0625     0.125
 0.25       0.25       0.125      0.125      0.03125    0.0078125
 0.00195312 0.        ]
[0.         0.00195312 0.00390625 0.015625   0.0625     